In [3]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
img = cv2.imread('/content/drive/MyDrive/TTNT/tiger.jpg')
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

rows,cols = img.shape[:2]

cv2_imshow(img)

ModuleNotFoundError: No module named 'cv2'

In [ ]:
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2_imshow(gray)

In [ ]:
kerrel_identity = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]])
output=cv2.filter2D(img,-1,kerrel_identity)
cv2_imshow(output)

In [ ]:
Egde_detection_kernel = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]])
output=cv2.filter2D(gray,-1,Egde_detection_kernel)
cv2_imshow(output)

Cau 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import neighbors, datasets
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
print ('Number of classes: %d' %len(np.unique(iris_y)))
print ('Number of data points: %d' %len(iris_y))
X0 = iris_X[iris_y == 0,:]
print ('\nSamples from class 0:\n', X0[:5,:])
X1 = iris_X[iris_y == 1,:]
print ('\nSamples from class 1:\n', X1[:5,:])
X2 = iris_X[iris_y == 2,:]
print ('\nSamples from class 2:\n', X2[:5,:])
X3 = iris_X[iris_y == 2,:]
print ('\nSamples from class 2:\n', X3[:5,:])
X4 = iris_X[iris_y == 2,:]
print ('\nSamples from class 2:\n', X4[:5,:])


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
 iris_X, iris_y, test_size=30)
print ("Training size: %d" %len(y_train))
print ("Test size : %d" %len(y_test))

In [ ]:
clf = neighbors.KNeighborsClassifier(n_neighbors = 10, p = 2)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print ("Print results for 20 test data points:")
print ("Predicted labels: ", y_pred[20:40])
print ("Ground truth : ", y_test[20:40])

In [ ]:
from sklearn.metrics import accuracy_score
print ("Accuracy of 1NN: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

In [ ]:
clf = neighbors.KNeighborsClassifier(n_neighbors = 10, p = 2)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print ("Accuracy of 10NN with major voting: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

In [ ]:
clf = neighbors.KNeighborsClassifier(n_neighbors = 10, p = 2, weights = 'distance')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print ("Accuracy of 10NN (1/distance weights): %.2f %%" %(100*accuracy_score(y_test, y_pred)))

In [ ]:
def myweight(distances):
 sigma2 = .5 # we can change this number
 return np.exp(-distances**2/sigma2)
clf = neighbors.KNeighborsClassifier(n_neighbors = 10, p = 2, weights = myweight)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print ("Accuracy of 10NN (customized weights): %.2f %%" %(100*accuracy_score(y_test, y_pred)))

Cau 3

In [ ]:
import cv2
import numpy as np
from skimage.feature import hog
#from sklearn.svm import LinearSVC
#from sklearn.svm import SVC
from sklearn.svm import SVC
svc = SVC()
from keras.datasets import mnist
from sklearn.metrics import accuracy_score
from google.colab.patches import cv2_imshow

In [ ]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()
 
X_train_feature = []
for i in range(len(X_train)):
    feature = hog(X_train[i],orientations=9,pixels_per_cell=(14,14),cells_per_block=(1,1),block_norm="L2")
    X_train_feature.append(feature)
X_train_feature = np.array(X_train_feature,dtype = np.float32)
 
#cho x_test
X_test_feature = []
for i in range(len(X_test)):
    feature = hog(X_test[i],orientations=9,pixels_per_cell=(14,14),cells_per_block=(1,1),block_norm="L2")
    X_test_feature.append(feature)
X_test_feature = np.array(X_test_feature,dtype=np.float32)

In [ ]:
model = SVC(C=10, probability=True)
model.fit(X_train_feature,y_train)
y_pre = model.predict(X_test_feature)
print(accuracy_score(y_test,y_pre))

In [ ]:
image = cv2.imread("/content/drive/MyDrive/TTNT/demo.jpg")
im_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
im_blur = cv2.GaussianBlur(im_gray,(5,5),0)
im,thre = cv2.threshold(im_blur,90,255,cv2.THRESH_BINARY_INV)
contours,hierachy = cv2.findContours(thre,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
rects = [cv2.boundingRect(cnt) for cnt in contours]
for i in contours:
    (x,y,w,h) = cv2.boundingRect(i)
   
    roi = thre[y:y+h,x:x+w]
    roi = np.pad(roi,(20,20),'constant',constant_values=(0,0))
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    roi = cv2.dilate(roi, (3, 3))
    
    # Calculate the HOG features
    roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1),block_norm="L2")
    nbr = model.predict(np.array([roi_hog_fd], np.float32))
    per = model.predict_proba(np.array([roi_hog_fd], np.float32))[0]
    per = float(str(round(per[nbr][0]*100,1)));
    print(per)

    if per > 80:
      cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),3)
      cv2.putText(image, str(int(nbr[0])), (x, y),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)
      cv2_imshow(image)
    else:
      cv2_imshow(image)
      print('ti le nhan dang thap hon 80')


cv2.waitKey()
cv2.destroyAllWindows()